# 介绍

一个常用的测试的notebook文件，用于查看想要查看的内容，或者一些测试代码等。

## 导包
通用导包，所有导包都放在这儿，之后测试一次性加载即可

In [1]:
import os
import sys
import math
import json
import time
import random

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import sklearn
import joblib
from tqdm import tqdm

# 获取项目根目录的绝对路径
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
print(project_root)
# 将项目根目录添加到 sys.path
if project_root not in sys.path:
    sys.path.append(project_root)
from common import utils


/mnt/nvme1/yihaoyuan/Raven/RavenEx/cross_modality_experiment/multimodal_emotion_recognition


In [2]:
checkpoint_path = "/mnt/nvme1/yihaoyuan/Raven/RavenEx/cross_modality_experiment/multimodal_emotion_recognition/logs/independent/HCI/2024-12-22_20-26-33/best_checkpoint/best_checkpoint_0.pth"
checkpoint = torch.load(checkpoint_path)
print(len(checkpoint))
# print(checkpoint[0])
# print(checkpoint[1].keys())
# print(type(checkpoint[0]))
# print(checkpoint[0].state_dict().keys())
# print(checkpoint[1].state_dict().keys())

for i in range(len(checkpoint)):
    print(type(checkpoint[i]))
    if isinstance(checkpoint[i], nn.Module):
        print(checkpoint[i].state_dict().keys())
    else:
        print(checkpoint[i])
        print(checkpoint[i].return_models)
        for model in checkpoint[i].return_models():
            print(model.state_dict().keys())
            print(model.state_dict().values())

5
<class 'models.CTFNPromptModels.ModalityAlign'>
odict_keys(['extractor.feature_global.gcn1.weight', 'extractor.feature_global.gcn1.bias', 'extractor.feature_global.gcn2.weight', 'extractor.feature_global.gcn2.bias', 'extractor.feature_global.gcn3.weight', 'extractor.feature_global.gcn3.bias', 'extractor.feature_global.pool.attn_gcn.weight', 'extractor.feature_global.pool.attn_gcn.bias', 'align.docking_0.weight', 'align.docking_0.bias', 'align.docking_1.weight', 'align.docking_1.bias', 'align.docking_2.weight', 'align.docking_2.bias', 'align.relative_position.0', 'align.relative_position.1', 'align.relative_position.2'])
<class 'models.CTFNPromptModels.DoubleTrans'>
<bound method DoubleTrans.return_models of <models.CTFNPromptModels.DoubleTrans object at 0x7fb3160e1fd0>>
odict_keys(['relative_pos', 'linear1.weight', 'linear1.bias', 'linear2.weight', 'linear2.bias', 'layers.0.self_attn.in_proj_weight', 'layers.0.self_attn.in_proj_bias', 'layers.0.self_attn.out_proj.weight', 'layers.0.s

In [15]:
print(checkpoint)

[ModalityAlign(
  (extractor): FeatureExtract(
    (feature_global): FeatureGlobal(
      (gcn1): GraphConvolution (75 -> 160)
      (gcn2): GraphConvolution (160 -> 160)
      (gcn3): GraphConvolution (160 -> 160)
      (pool): SelfAttentionPooling(
        (attn_gcn): GraphConvolution (480 -> 1)
      )
    )
  )
  (align): Embrace(
    (docking_0): Linear(in_features=960, out_features=160, bias=True)
    (docking_1): Linear(in_features=41, out_features=160, bias=True)
    (docking_2): Linear(in_features=119, out_features=160, bias=True)
    (relative_position): ParameterList(
        (0): Parameter containing: [torch.float32 of size 1x1x160 (GPU 0)]
        (1): Parameter containing: [torch.float32 of size 1x1x160 (GPU 0)]
        (2): Parameter containing: [torch.float32 of size 1x1x160 (GPU 0)]
    )
  )
), EmotionClassificationModel(
  (encoder): AttentionEncoder(
    (encoder_layer): ModuleList(
      (0): EncoderLayer(
        (norm_1): Norm()
        (norm_2): Norm()
        (

In [4]:
test = [nn.Parameter(torch.zeros(1,10,160)) for _ in range(10)]
print(test)

[Parameter containing:
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], requires_grad=True), Parameter containing:
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], requires_grad=True), Parameter containing:
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], requires_grad=True), Parameter containing:
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,

In [ ]:
# 之前的node features形状
eeg_path = "/data/Ruiwen/ExtractedFeatures2/node_features_combine.npy"
eeg = np.load(eeg_path)
print(eeg.shape)

(50592, 150)


In [23]:
# 现有的用来分析的数据形式如下
ruiwen_path = "/data/Ruiwen/data_with_ICA.pkl"  # 只包含脑电和眼动，不包含人脸
ruiwen_data = joblib.load(ruiwen_path)
print(ruiwen_data.keys())
print(ruiwen_data["info"])
print(ruiwen_data['ch_info'])

subject_lists = ruiwen_data['subject_list']
print(subject_lists)

dict_keys(['label', 'subject_list', 'ch_info', 'info', 'eye_info', 'eeg', 'eye', 'au'])
EEG: 1-70filer, 50Hz notch, With ICA, 256Hz resample;
Subject : 1-34 subject, no 1,23,32 subject, 15 subject exists eye data missing; 31 person, 48 question, 31 channel;
Labels : 0: Confused,1: Guess, 2:Unconfused, 4: Think-right;
Eye Track data : Pupil diameter left, Pupil diameter right,Gaze point X, Gaze point Y, Eye movement type, Gaze event duration
['Fp1', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2']
[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34]


In [24]:
eeg_data = ruiwen_data['eeg']
print(len(eeg_data))

31


In [ ]:
au_feature = ruiwen_data['au']
print(len(au_feature))


31


In [28]:
for i, subject_list in tqdm(enumerate(subject_lists)):
    trials = eeg_data[i]


31it [00:00, 807598.91it/s]


In [33]:
# 眼动特征
eye_data = ruiwen_data['eye']
print(type(eye_data))
for per in eye_data:
    print(type(per))
    print(len(per))
    print(per[0].shape)


<class 'list'>
<class 'list'>
48
(433, 6)
<class 'list'>
48
(418, 6)
<class 'list'>
48
(596, 6)
<class 'list'>
48
(349, 6)
<class 'list'>
48
(855, 6)
<class 'list'>
48
(389, 6)
<class 'list'>
48
(507, 6)
<class 'list'>
48
(830, 6)
<class 'list'>
48
(617, 6)
<class 'list'>
48
(655, 6)
<class 'list'>
48
(875, 6)
<class 'list'>
48
(854, 6)
<class 'list'>
48
(697, 6)
<class 'list'>
48
(730, 6)
<class 'list'>
48
(673, 6)
<class 'list'>
48
(902, 6)
<class 'list'>
48
(560, 6)
<class 'list'>
48
(848, 6)
<class 'list'>
48
(825, 6)
<class 'list'>
48
(485, 6)
<class 'list'>
48
(875, 6)
<class 'list'>
48
(579, 6)
<class 'list'>
48
(652, 6)
<class 'list'>
48
(581, 6)
<class 'list'>
48
(660, 6)
<class 'list'>
48
(638, 6)
<class 'list'>
48
(442, 6)
<class 'list'>
48
(324, 6)
<class 'list'>
48
(735, 6)
<class 'list'>
48
(904, 6)
<class 'list'>
48
(472, 6)


In [37]:
data_path = "/data/Ruiwen/data_with_ICA.pkl"
print(utils.find_nearest_folder(data_path))

/data/Ruiwen


In [38]:
eye_features_path = "/data/Ruiwen/eye_track_feature"
for id in subject_lists:
    path = os.path.join(eye_features_path, f"{str(id)}.npy")
    feature = np.load(path)
    print(feature.shape)

(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)
(48, 41)



## 修改数据key，此外把AU添加其中

In [19]:
# 现有的用来分析的数据形式如下
ruiwen_path = "/data/Ruiwen/data_with_ICA.pkl"  # 只包含脑电和眼动，不包含人脸
ruiwen_data = joblib.load(ruiwen_path)
print(ruiwen_data.keys())
print(ruiwen_data["info"])
print(ruiwen_data['ch_info'])
# print(ruiwen_data.info)

# 首先修改一下key，然后把AU特征也添加其中
ruiwen_data['eeg'] = ruiwen_data.pop('eeg_data')
ruiwen_data['eye'] = ruiwen_data.pop('eye_track_data')
print(ruiwen_data.keys())


dict_keys(['eeg_data', 'eye_track_data', 'label', 'subject_list', 'ch_info', 'info', 'eye_info'])
EEG: 1-70filer, 50Hz notch, With ICA, 256Hz resample;
Subject : 1-34 subject, no 1,23,32 subject, 15 subject exists eye data missing; 31 person, 48 question, 31 channel;
Labels : 0: Confused,1: Guess, 2:Unconfused, 4: Think-right;
Eye Track data : Pupil diameter left, Pupil diameter right,Gaze point X, Gaze point Y, Eye movement type, Gaze event duration
['Fp1', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'TP9', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'TP10', 'CP6', 'CP2', 'Cz', 'C4', 'T8', 'FT10', 'FC6', 'FC2', 'F4', 'F8', 'Fp2']
dict_keys(['label', 'subject_list', 'ch_info', 'info', 'eye_info', 'eeg', 'eye'])


In [20]:

# 加载AU特征
subject_lists = ruiwen_data['subject_list']
print(subject_lists)
print(len(subject_lists))
print(type(subject_lists))
au_path = "/data/Ruiwen/au_feature"
au_features = []
for subject in subject_lists:
    au_feature = np.load(os.path.join(au_path, str(subject) + ".npy"))
    # print(au_feature.shape)
    au_features.append(au_feature)
au_features = np.array(au_features)
print(au_features.shape)
ruiwen_data['au'] = au_features
print(ruiwen_data.keys())

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34]
31
<class 'list'>
(31, 48, 119)
dict_keys(['label', 'subject_list', 'ch_info', 'info', 'eye_info', 'eeg', 'eye', 'au'])


In [21]:
# 保存新的数据
ruiwen_path = "/data/Ruiwen/data_with_ICA.pkl"
joblib.dump(ruiwen_data, ruiwen_path)

['/data/Ruiwen/data_with_ICA.pkl']

In [5]:
eeg_data = ruiwen_data["eeg_data"]  # 31，48,（sample_len, ch_nums)
print(len(eeg_data))
print(len(eeg_data[0]))
print(len(eeg_data[0][0]))
print(len(eeg_data[0][0][0]))
print(len(eeg_data[0][0][0]))

31
48
1847
31
31


In [ ]:
# 之前用于训练的数据加载如下
data_path = "/data/Ruiwen"
print("\n".join(os.listdir(data_path)))
eeg_path = os.path.join(data_path, "ExtractedFeatures2", "node_features_combine.npy")
eye_dir_path = os.path.join(data_path, "eye_track_feature")
face_dir_path = os.path.join(data_path, "au_feature")

eeg_data = np.load(eeg_path)


ExtractedData
adjacency_list_20231217.mat
adjacency
Raven.zip
adjacency_list_9.mat
ExtractedFeatures2
adjacency_list_8.mat
data.pkl
zhangyi
adjacency_list.mat
adjacency_list_20240425.mat
adjacency_list_14.mat
eye_track_feature
adjacency_list_10.mat
openface
ExtractedFeatures
adjacency_list_test.mat
class_all.csv
Raven
dict_data.npy
au_feature
adjacency_list_multi_all.mat
adjacency_list_20231214.mat
data_with_ICA.pkl
原始数据
眼动数据分割.rar
adjacency_list_16.mat
adjacency_list_15.mat
eeg_feature


In [ ]:
# 

# 测试自助法和SMOTE来解决类别不平衡问题

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.utils import resample

# 创建一个不均衡的分类数据集（类别0远多于类别1）
X = np.array([[1], [2], [3], [4], [5], [6], [7], [8], [9], [10],  # 特征
              [11], [12], [13], [14], [15], [16], [17], [18], [19], [20]])  
y = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  # 类别0的样本
              1, 1, 1, 0, 0, 0, 0, 0, 1, 1])  # 类别1的样本（类别1非常少）

# 查看类别分布
print(f"Original class distribution: {pd.Series(y).value_counts()}")

# 1. 使用自助法生成多个训练集（生成 5 个训练集）
num_samples = 5
bootstrap_samples = []
bootstrap_labels = []

for _ in range(num_samples):
    # 抽取自助样本（有放回抽样）
    indices = np.random.choice(len(X), size=len(X), replace=True)
    X_sample, y_sample = X[indices], y[indices]
    
    # 确保少数类样本数量足够进行SMOTE
    minority_class_count = (y_sample == 1).sum()
    if minority_class_count < 2:
        print(f"Warning: Sample size for minority class too small in bootstrap sample. Skipping SMOTE for this sample.")
        continue  # 如果少数类样本数过少，跳过该自助样本

    # 2. 对每个自助样本使用SMOTE进行少数类过采样
    smote = SMOTE(sampling_strategy='auto', random_state=42, k_neighbors=3)  # 设置k_neighbors为小于少数类样本数的值
    X_resampled, y_resampled = smote.fit_resample(X_sample, y_sample)
    
    # 存储自助样本和对应的标签
    bootstrap_samples.append(X_resampled)
    bootstrap_labels.append(y_resampled)

# 查看自助样本的类别分布
print(f"\nClass distribution in resampled bootstrap samples:")
for i in range(num_samples):
    print(f"Bootstrap sample {i+1} class distribution: {pd.Series(bootstrap_labels[i]).value_counts()}")

# 3. 在每个自助样本上训练模型并评估性能
for i in range(len(bootstrap_samples)):  # 只在有效的自助样本上训练
    X_sample = bootstrap_samples[i]
    y_sample = bootstrap_labels[i]
    
    # 训练决策树分类器
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_sample, y_sample)
    
    # 预测并计算准确率
    y_pred = clf.predict(X)
    accuracy = accuracy_score(y, y_pred)
    
    print(f"\nAccuracy of model trained on bootstrap sample {i+1}: {accuracy}")


Original class distribution: 0    15
1     5
dtype: int64

Class distribution in resampled bootstrap samples:
Bootstrap sample 1 class distribution: 1    16
0    16
dtype: int64
Bootstrap sample 2 class distribution: 0    13
1    13
dtype: int64
Bootstrap sample 3 class distribution: 1    12
0    12
dtype: int64
Bootstrap sample 4 class distribution: 0    15
1    15
dtype: int64
Bootstrap sample 5 class distribution: 1    14
0    14
dtype: int64

Accuracy of model trained on bootstrap sample 1: 0.9

Accuracy of model trained on bootstrap sample 2: 0.9

Accuracy of model trained on bootstrap sample 3: 0.9

Accuracy of model trained on bootstrap sample 4: 1.0

Accuracy of model trained on bootstrap sample 5: 0.95
